# 多动作单智能体

## **具有多个动作的智能体**

我们注意到一个智能体能够执行一个动作，但如果只有这些，实际上我们并不需要一个智能体。通过直接运行动作本身，我们可以得到相同的结果。智能体的力量，或者说Role抽象的惊人之处，在于动作的组合（以及其他组件，比如记忆，但我们将把它们留到后面的部分）。通过连接动作，我们可以构建一个工作流程，使智能体能够完成更复杂的任务。

假设现在我们不仅希望用自然语言编写代码，而且还希望生成的代码立即执行。一个拥有多个动作的智能体可以满足我们的需求。让我们称之为RunnableCoder，一个既写代码又立即运行的Role。我们需要两个Action：SimpleWriteCode 和 SimpleRunCode。

## **定义动作**

首先，定义 SimpleWriteCode。我们将重用上面创建的那个。

接下来，定义 SimpleRunCode。如前所述，从概念上讲，一个动作可以利用LLM，也可以在没有LLM的情况下运行。在SimpleRunCode的情况下，LLM不涉及其中。我们只需启动一个子进程来运行代码并获取结果。我们希望展示的是，对于动作逻辑的结构，我们没有设定任何限制，用户可以根据需要完全灵活地设计逻辑。



In [1]:
# SimpleWriteCode 这个类与上一节一模一样

from metagpt.actions import Action

class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

2025-02-26 14:31:59.312 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /tmp/code/wow-agent/notebooks


In [2]:
# 本节新增了SimpleRunCode这个类
class SimpleRunCode(Action):
    name: str = "SimpleRunCode"

    async def run(self, code_text: str):
        result = subprocess.run(["python", "-c", code_text], capture_output=True, text=True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

## **定义角色**

与定义单一动作的智能体没有太大不同！让我们来映射一下：

1. 用 self.set\_actions 初始化所有 Action
2. 指定每次 Role 会选择哪个 Action。我们将 react\_mode 设置为 "by\_order"，这意味着 Role 将按照 self.set\_actions 中指定的顺序执行其能够执行的 Action。在这种情况下，当 Role 执行 \_act 时，self.rc.todo 将首先是 SimpleWriteCode，然后是 SimpleRunCode。
3. 覆盖 \_act 函数。Role 从上一轮的人类输入或动作输出中检索消息，用适当的 Message 内容提供当前的 Action (self.rc.todo)，最后返回由当前 Action 输出组成的 Message。



In [3]:
import re
import os
import subprocess
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger
class RunnableCoder(Role):
    name: str = "Alice"
    profile: str = "RunnableCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteCode, SimpleRunCode])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0]  # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=result, role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg

In [4]:
async def main():
    msg = "write a function that calculates the sum of a list"
    role = RunnableCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)
    return result

rtn = await main()
print(rtn)

2025-02-26 14:32:41.080 | INFO     | __main__:main:4 - write a function that calculates the sum of a list
2025-02-26 14:32:41.083 | INFO     | __main__:_act:17 - Alice(RunnableCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def sum_list(numbers):
    return sum(numbers)

# Test cases
def test_case_1():
    assert sum_list([1, 2, 3, 4, 5]) == 15

def test_case_2():
    assert sum_list([]) == 0
```

2025-02-26 14:32:44.236 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.
2025-02-26 14:32:44.241 | INFO     | __main__:_act:17 - Alice(RunnableCoder): to do SimpleRunCode(SimpleRunCode)
2025-02-26 14:32:44.276 | INFO     | __main__:run:8 - code_result=''
2025-02-26 14:32:44.279 | INFO     | __main__:main:6 - RunnableCoder: 



RunnableCoder: 
